# 오차역전파법

## 단순한 계층 구현

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out
    
    def backward(self,dout):
        dx = dout * self.y #x와 y를 바꾼다
        dy = dout * self.x

        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
#역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


## 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass #덧셈 계층에서는 초기화가 필요 없어서 pass

    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

#역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 활성화 함수 계층 구현하기

In [6]:
# ReLU 계층

class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0) # x값이 0 이하이면
        out = x.copy()
        out[self.mask] = 0 # 결과는 0이고

        return out
    
    def backward(self, dout):
        dout[self.mask] = 0 # 역전파 값도 0
        dx = dout

        return dx

In [7]:
import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])

print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [8]:
mask = (x<=0)
print(mask)

[[False  True]
 [ True False]]


In [9]:
# sigmoid 계층

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1+np.exp(-x))
        self.out = out

        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## Affine(행렬곱)/Softmax 계층 구현하기

In [10]:
X_dot_W = np.array([[0,0,0], [10,10,10]])

B = np.array([1,2,3])

X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [11]:
dY = np.array([[1,2,3],[4,5,6]])

dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

In [12]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = B
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b # 행렬곱

        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

### softmax-with-loss 계층(입력값을 정규화하여 출력의 합이 1이 되도록 확률값으로 출력)

In [17]:
import sys, os
sys.path.append(os.pardir)

from common.functions import *
from common.util import im2col, col2im

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None # softmax의 출력
        self.t = None # 정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

        return dx

In [19]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):

        # 가중치 초기화

        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(hidden_size)

        # 계층 생성

        self.layers = OrderedDict() #순서가 있는 딕셔너리로 저장하여 순서대로 각 계층의 forward를 호출하고 역전파는 반대로 호출
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x
    
    def loss(self, x, t): # x:입력 데이터, t:정답 레이블
        y = self.predict(x)
        return self.lastLayer.forward(y,t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1) #예측 결과 y에서 가장 높은 값을 가진 인덱스를 반환
        if t.ndim != 1: 
            t = np.argmax(t, axis=1) 
            #t가 1차원 배열이 아니라면, 다중 클래스 형식(예: 원-핫 인코딩)으로 주어진 것이므로, 
            #np.argmax(t, axis=1)을 사용하여 각 데이터 포인트에 대한 실제 클래스를 가져옴 -> t도 예측과 같은 형식(1차원 배열)으로 변환

        accuracy = np.sum(y==t) / float(x.shape[0])

        return accuracy
    

    def numerical_gradient(self, x, t):
        loss_W = lambda W : self.loss(x,t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads
    
    def gradient(self, x, t):
        #순전파
        self.loss(x, t)

        #역전파
        dout = 1 #최종 손실에 대한 미분값을 1로 설정(dout을 1로 두면 그 결과가 뒤따르는 모든 층에 대해 기울기가 그대로 유지)
        dout = self.lastLayer.backward(dout) 

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads


### 오차역전파법으로 구한 기울기 검증하기

In [20]:
# 수치 미분(구현 쉬움, 오류 적음) 과 오차역전파법(구현 복잡, 버그 많음)의 결과를 비교하여 결과 검증

import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from ch04.two_layer_net import TwoLayerNet


# 데이터 읽기
(x_train, t_train) , (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch) # 수치 미분으로 구한 기울기
grad_backprop = network.gradient(x_batch, t_batch) # 오차역전파법으로 구한 기울기

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + " : " + str(diff))

## 결과 해석
# 오차값이 적으므로 결과가 서로 유사하다는 의미

W1 : 4.698581075600372e-13
b1 : 1.2309856384012594e-12
W2 : 1.4912512877329755e-11
b2 : 1.1945999189855172e-10


### 오차역전파법을 사용한 학습 구현하기

In [21]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from ch04.two_layer_net import TwoLayerNet


# 데이터 읽기
(x_train, t_train) , (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size = 784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size) #주어진 범위(여기서는 train_size = 훈련 데이터의 총 개수)에서 랜덤하게 batch_size만큼의 인덱스를 선택
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    #오차역전파법으로 기울기 구하기
    grad = network.gradient(x_batch, t_batch)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0 : # i를 iter_per_epoch로 나눴을 때 나머지가 0일때 (즉, i가 iter_per_epoch의 배수) = 모델의 성능을 주기적으로 모니터링하기 위함
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)


0.09751666666666667 0.0974
0.7991333333333334 0.8026
0.8767833333333334 0.8784
0.8971833333333333 0.9009
0.90795 0.9085
0.9147666666666666 0.9171
0.9197166666666666 0.9215
0.9244 0.9263
0.9271166666666667 0.9282
0.9312333333333334 0.9306
0.9338 0.9339
0.9359 0.9364
0.93935 0.9386
0.9411833333333334 0.9397
0.9428666666666666 0.9407
0.9455333333333333 0.9429
0.9472 0.9443
